# Attention & Transformer
**어텐션(Attention)**과 **트랜스포머(Transformer)**는 자연어 처리 분야에서 굉장히 활발하게 사용되고 있는 모델입니다.</br>
두 모델을 사용하면 영어를 한국어로 바꾸어주는 번역기나, 질문에 대한 답을 제공하는 챗봇 등을 어렵지 않게 만들어 낼 수 있습니다.

해당 노트에서는 **어텐션과 트랜스포머 모델의 구조와 원리**에 대해 알아보고 간단한 **한국어 챗봇을 구현**해보도록 하겠습니다.

## 필요 라이브러리

In [ ]:
import tensorflow as tf
import pandas as pd
import re
import tensorflow_datasets as tfds

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

어텐션과 트랜스포머 모델에 대해 알아보기 전에 문장 생성의 가장 기본적인 모델인 **Seq2Seq**부터 알아보도록 하겠습니다.

## **1. Seq2Seq**
시퀀스 투 시퀀스(Sequence-to-Sequence) 모델은 **입력된 도메인의 정보를 바탕으로 다른 도메인의 결과를 출력하는 모델**입니다.</br>
모델이 가지고 있는 특징때문에 챗봇, 기계번역 등 굉장히 다양한 분야에서 사용되고 있습니다.

### **1-1. 모델 구조**
Seq2Seq 모델은 크게 **인코더(Encoder), 컨텍스트 벡터(Context vector),  디코더(Decoder)** 3가지로 이루어져 있습니다.</br>

<img src="https://i.imgur.com/yCppt05.png" alt="seq2seq_6" width="800" />

#### **1) 인코더**
- 인코더는 **입력 정보의 특징을 추출하는 모듈**입니다.</br>
- 인코더 내부에 있는 RNN, LSTM, GRU와 같은 순환 신경망 모델을 통해 입력 데이터의 특징을 추출하고,</br> 하나의 컨텍스트 벡터로 만들어 디코더에 전달합니다.</br>
- [입력값(Input)] : 번역하고자 하는 문장(기계번역) 또는 질문(챗봇) 등을 입력으로 받습니다.
- [출력값(Output)] : 입력 데이터에서 추출한 특징을 담은 컨텍스트 벡터를 생성합니다.</br>
<img src="https://i.imgur.com/BofLfSi.png" alt="seq2seq_6" width="400" />

#### **2) 디코더**
- 디코더는 입력 정보를 토대로 **새로운 정보를 생성하는 모듈**입니다.</br>
- [입력값(Input)] : 인코더에서 생성한 컨텍스트 벡터와 현재 생성하고 있는 출력 정보 </br>
- [출력값(Output)] : 순차적으로 단어 생성 </br>
<img src="https://i.imgur.com/YTTVW3e.png" alt="seq2seq_6" width="400" />

#### **3) 컨텍스트 벡터**
- **입력 데이터의 특징을 모두 담고있는 벡터.**</br>
- Seq2seq에서는 인코더의 가장 마지막 LSTM(혹은 RNN)에서 생성한 **hidden state 벡터 및 cell state 벡터**가 된다. </br>
- 디코더의 첫 LSTM에 들어가는 hidden state 벡터 및 cell state 벡터

### **1-2. 모델 구현**
인코더와 디코더를 구현하고 하나로 합쳐 Seq2Seq 모델을 만들어보도록 하겠습니다.

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        '''
        Seq2Seq의 인코더입니다.

        Args:
            units (int) : 인코더 내부 lstm의 노드 수.
            vocab_size (int) : 임베딩 행렬의 단어 수. 없는 단어가 있으면 oov가 발생할 수 있습니다.
                -> 훈련하려는 문장의 단어는 모두 포함하고 있는 것이 좋다!
            embedding_dim (int) : 임베딩 차원 수. 복잡할수록 좋을 수도 있고, 아닐 수도 있습니다. 차원이 크면 보통 표현력이 좋다. but 용량이 커져서 안좋을 수 있다.
            time_steps (int) : 문장 토큰의 수. ex) 안녕하세요 조윤행입니다. -> 안녕하세요/ 조윤행/ 입니다/ -> 토큰 수 : 3개
        '''
        super().__init__()
        self.embedding = Embedding(vocab_size,
                                   embedding_dim,
                                   input_length=time_steps)
        self.dropout = Dropout(0.2)
        self.lstm = LSTM(units, return_state=True)

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x)
        return [hidden_state, cell_state]

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        '''
        Seq2Seq의 디코더입니다.

        Args:
            units (int) : 디코더 내부 lstm의 노드 수.
            vocab_size (int) : 임베딩 행렬의 단어 수. 없는 단어가 있으면 oov가 발생할 수 있습니다.
                -> 훈련하려는 문장의 단어는 모두 포함하고 있는 것이 좋다!
            embedding_dim (int) : 임베딩 차원 수. 복잡할수록 좋을 수도 있고, 아닐 수도 있습니다. 차원이 크면 보통 표현력이 좋다. but 용량이 커져서 안좋을 수 있다.
            time_steps (int) : 문장 토큰의 수. 디코더에서는 최대로 생성할 수 있는 문장의 길이가 됩니다.
        '''
        super().__init__()
        self.embedding = Embedding(vocab_size,
                                   embedding_dim,
                                   input_length=time_steps)
        self.dropout = Dropout(0.2)
        self.lstm = LSTM(units,
                         return_state=True,
                         return_sequences=True)
        self.dense = Dense(vocab_size, activation='softmax')

    def call(self, inputs, initial_state):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)
        x = self.dense(x)
        return x, hidden_state, cell_state

In [ ]:
class Seq2seq(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
        """
        Seq2Seq 모델입니다. 인코더와 디코더를 선언합니다.
        """
        super().__init__()
        self.start_token = start_token
        self.end_token = end_token
        self.time_steps = time_steps

        self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
        self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)

    def call(self, inputs, training=True):
        """
        선언한 인코더와 디코더를 하나로 연결하여 Seq2Sqe 파이프라인을 구현합니다.

        Args:
            inputs : 문장의 단어 인덱스로 이루어진 데이터.
            training : True인 경우 교사강요를 사용하여 디코더의 입력값에 정답을 넣어주며, False인 경우 디코더의 출력 데이터를 입력으로 넣어주게 됩니다.
        """
        if training:
            encoder_inputs, decoder_inputs = inputs
            context_vector = self.encoder(encoder_inputs)
            decoder_outputs, _, _ = self.decoder(decoder_inputs, context_vector)
            return decoder_outputs
        else:
            context_vector = self.encoder(inputs)
            target_seq = tf.constant([[self.start_token]], dtype=tf.float32)
            results = tf.TensorArray(tf.int32, self.time_steps)

            for i in tf.range(self.time_steps):
                decoder_output, decoder_hidden, decoder_cell = self.decoder(target_seq,
                                                                            context_vector)
                decoder_output = tf.cast(tf.argmax(decoder_output, axis= -1),
                                         dtype=tf.int32)
                decoder_output = tf.reshape(decoder_output, shape=(1, 1))
                results = results.write(i, decoder_output)

                if decoder_output == self.end_token:
                    break

                target_seq = decoder_output
                context_vector = [decoder_hidden, decoder_cell]

            return tf.reshape(results.stack(), shape=(1, self.time_steps))

### **1-3. 챗봇 만들기**
제작한 Seq2Seq 모델을 사용하여 간단한 훈련을 통해 챗봇을 만들어보도록 하겠습니다.

#### **1) 데이터 로드 및 전처리**
한국어 챗봇 제작을 위해 Korpora의 한국어 문답 데이터를 사용해보도록 하겠습니다.</br>
Github : https://github.com/ko-nlp/Korpora

In [ ]:
#API를 사용하는 경우 pip 오류가 발생하므로 csv 형태의 Raw데이터를 그대로 가져와 사용하겠습니다.
corpus = pd.read_csv('https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv')

# 2,000개 데이터 셋만 활용 (Google Colab 일 경우 3,000개에서는 메모리 오버되는 현상 발생)
texts = []
pairs = []
for i, (text, pair) in enumerate(zip(corpus['Q'], corpus['A'])):
    texts.append(text)
    pairs.append(pair)
    if i >= 2000:
        break

# 데이터 체크
list(zip(texts, pairs))[1995:2000]

[('배 아프다', '약이 필요하면 도움을 받아보세요.'),
 ('배 터지겠네', '위를 좀 쉬게 해주세요.'),
 ('배 터지겠다.', '산책 좀 해야겠네여.'),
 ('배가 너무 고파', '뭐 좀 챙겨드세요.'),
 ('배가 넘넘 고파', '저도 밥 먹고 싶어요')]

데이터를 확인했을 때, 마침표와 같은 기호가 있는 경우가 있고, 없는 경우가 있으므로 제거해주도록 합시다.</br>
물음표와 공백의 경우 문장 생성의 중요한 특징이 될 수 있으므로 제거하지 않습니다.

In [ ]:
#regular expression(regex)를 사용
import re

def cleaning_sent(sentence):
    '''
    한글 및 숫자, 물음표 및 공백을 제외하고 제거하는 함수입니다.

    Input:
        sentence : str. 정제하려는 문장을 입력으로 받습니다.
    Return:
        정제 완료된 문장 반환. str
    '''
    sent = re.sub(r'[^1-9가-힣? ]', '', sentence)
    return sent

In [ ]:
# 정제 함수 테스트
cleaning_sent('오늘은 뭐해?')

'오늘은 뭐해?'

한국어를 사용하므로 한국어 전용 형태소 분석기인 Konlpy를 사용하도록 하겠습니다.

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 465 kB 73.5 MB/s 


Konlpy에서는 한국어 품사를 태그해주는 다양한 태거를 지원하고 있습니다.</br>
이 중에서 이번에는 트위터를 기반으로 만들어진 Okt를 활용하도록 하겠습니다.</br>
Konlpy 형태소 분석 및 품사 태깅 공식 문서 : https://konlpy.org/ko/v0.6.0/morph/

In [ ]:
from konlpy.tag import Okt
okt = Okt()

def morpheme_anlysis(sentence):
    '''
    문장이 들어왔을 때, 형태소 단위로 분석해주는 함수입니다.
    Input:
        sentence : str. 형태소 분석을 하려는 문장입니다.
    Return:
        형태소 단위로 뛰워쓰기 되어있는 문장 반환. str
    '''
    return ' '.join(okt.morphs(sentence))

인코더와 디코더의 입력값, 출력값에 따라 문장의 형태가 조금씩 다릅니다. </br>
따라서 각각에 맞는 형태로 문장을 바꾸어주는 함수를 제작합니다.

질문 : 인코더의 Input</br>
대답 : 디코더의 Input, Output

In [ ]:
def clean_and_morph(sentence, is_question=True):
    '''
    문장이 질문인지 대답인지에 맞추어 알맞은 형태로 변형해주는 함수입니다.

    Input:
        sentence : str. 변형하려는 문장입니다.
        is_question : 문장이 질문인지, 대답인지 알려주는 값입니다. 문장이면 True, 대답이면 False를 받습니다.

    Return:
        문장 타입에 따라 알맞게 변형된 문장을 반환합니다.
    '''

    sentence = cleaning_sent(sentence)
    sentence = morpheme_anlysis(sentence)

    if is_question:
        return sentence
    else:
        return ('<sos> ' + sentence, sentence + ' <eos>')

def preprocessing(questions, pairs):
    '''
    한국어 문답 데이터를 전처리하는 함수입니다.
    정제, 형태소 분석, 모델의 인풋,아웃풋으로 사용할 수 있는 형태로 변환하는 과정을 포함하고 있습니다.

    Input:
        questions : list. 문답 데이터 중 질문으로 이루어진 리스트를 받습니다.
        pairs : list. 문답 데이터 중 답변으로 이루어진 리스트를 받습니다.

    Return:
        질문, 인풋 형태의 문답, 아웃풋 형태의 문답
    '''
    answer_in = []
    answer_out = []

    pre_questions = [clean_and_morph(question, is_question=True) for question in questions]

    for pair in pairs:
        in_pair, out_pair = clean_and_morph(pair, is_question=False)
        answer_in.append(in_pair)
        answer_out.append(out_pair)

    return pre_questions, answer_in, answer_out

questions, answer_in, answer_out = preprocessing(texts, pairs)

In [ ]:
#변환 확인
print(questions[:3])
print(answer_in[:3])
print(answer_out[:3])

['12시 땡', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다']
['<sos> 하루 가 또 가네요', '<sos> 위로 해 드립니다', '<sos> 여행 은 언제나 좋죠']
['하루 가 또 가네요 <eos>', '위로 해 드립니다 <eos>', '여행 은 언제나 좋죠 <eos>']


#### **2) 토큰화**
전처리가 완료된 데이터를 모델에 넣을 수 있는 형태로 토큰화합니다.

In [ ]:
#토크나이저 학습을 위해 전체 데이터 리스트를 하나 생성
all_sentence = questions + answer_in + answer_out
len(all_sentence)

6003

In [ ]:
#리스트를 사용하여 tokenizer 학습
tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')
tokenizer.fit_on_texts(all_sentence)

In [ ]:
#토큰 인덱스 확인
for word, idx in tokenizer.word_index.items():
    print(f'{word}\t -> \t{idx}')
    if idx > 10:
        break

<OOV>	 -> 	1
<sos>	 -> 	2
<eos>	 -> 	3
이	 -> 	4
을	 -> 	5
거	 -> 	6
가	 -> 	7
예요	 -> 	8
도	 -> 	9
해보세요	 -> 	10
요	 -> 	11


In [ ]:
#질문 답변 데이터 토큰화
question_sequence = tokenizer.texts_to_sequences(questions)
answer_in_sequence = tokenizer.texts_to_sequences(answer_in)
answer_out_sequence = tokenizer.texts_to_sequences(answer_out)

In [ ]:
#토큰화가 잘 적용되었는지 확인
print(questions[:3])
print(question_sequence[:3])

['12시 땡', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다']
[[1758, 2493], [1609, 2494, 2495, 1610], [974, 1759, 1760, 213, 197, 106]]


In [ ]:
#문장 최대 토큰 수 계산
#max_len은 모델이 생성할 수 있는 문장의 최대 길이도 되므로 고려해서 선정해야함
def len_cal(sentences):
    total_len = 0
    count = 0
    maxlen = 0

    for i in sentences:
        if maxlen < len(i):
            maxlen = len(i)
        total_len += len(i)
        count += 1

    print(total_len / count)
    print(maxlen)

len_cal(question_sequence)
len_cal(answer_in_sequence)
len_cal(answer_out_sequence)

4.228385807096452
12
5.888055972013993
20
5.888055972013993
20


In [ ]:
#패딩
max_len = 20
question_pad = pad_sequences(question_sequence,
                             max_len,
                             padding= 'post')
answer_in_pad = pad_sequences(answer_in_sequence,
                             max_len,
                             padding= 'post')
answer_out_pad = pad_sequences(answer_out_sequence,
                             max_len,
                             padding= 'post')

In [ ]:
#패딩 적용 확인
question_pad.shape, answer_in_pad.shape, answer_out_pad.shape

((2001, 20), (2001, 20), (2001, 20))

encoder & decoder

In [ ]:
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM
import numpy as np

#### **3) 모델 훈련**
데이터와 모델이 모두 준비되었으므로, 한국어 문답 데이터로 모델을 훈련합니다.

현재 Seq2Seq 모델은 출력 결과가 인덱스로 이루어져있으므로, 이를 문장으로 바꿔주는 함수를 제작합니다.

In [ ]:
#인덱스를 단어로 변환하는 함수
def convert_index_to_text(indexs, end_token):

    sentence = ''

    for index in indexs:
        if index == end_token:
            break;
        if index > 0 and tokenizer.index_word[index] is not None:
            sentence += tokenizer.index_word[index]
        else:
            sentence += ''

        sentence += ' '
    return sentence

In [ ]:
#문장을 입력받으면 대답을 만들어내는 함수를 제작합니다.
def make_prediction(model, question_inputs):
    results = model(inputs=question_inputs, training=False)
    # 변환된 인덱스를 문장으로 변환
    results = np.asarray(results).reshape(-1)
    return results

In [ ]:
#학습 파라미터
#하이퍼 파라미터 튜닝 시 참고하기 쉽도록 한 셀에 모아 작성하는 것이 좋습니다.

BUFFER_SIZE = 1000
BATCH_SIZE = 16
EMBEDDING_DIM = 100
TIME_STEPS = max_len
START_TOKEN = tokenizer.word_index['<sos>']
END_TOKEN = tokenizer.word_index['<eos>']

UNITS = 128

#padding을 포함하기위해 +1
VOCAB_SIZE = len(tokenizer.word_index) + 1
DATA_LENGTH = len(questions)
SAMPLE_SIZE = 5
NUM_EPOCHS = 20

In [ ]:
#모델 훈련을 위해 모델을 선언합니다.
seq2seq = Seq2seq(UNITS,
                  VOCAB_SIZE,
                  EMBEDDING_DIM,
                  TIME_STEPS,
                  START_TOKEN,
                  END_TOKEN)

seq2seq.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['acc'])

In [ ]:
#학습된 모델을 저장하기위해 체크포인트를 지정합니다.
checkpoint_path = 'model/seq2seq.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only= True,
                             save_best_only=True,
                             monitor='loss',
                             verbose=1
                             )

In [ ]:
#모델 훈련
for epoch in range(NUM_EPOCHS):
    print(f'processing epoch: {epoch * 10 + 1}...')
    seq2seq.fit([question_pad, answer_in_pad],
                answer_out_pad,
                epochs=10,
                batch_size=BATCH_SIZE,
                callbacks=[checkpoint]
               )
    # 랜덤한 샘플 번호 추출
    samples = np.random.randint(DATA_LENGTH, size=SAMPLE_SIZE)

    # 예측 성능 테스트
    for idx in samples:
        question_inputs = question_pad[idx]
        # 문장 예측
        results = make_prediction(seq2seq, np.expand_dims(question_inputs, 0))

        # 변환된 인덱스를 문장으로 변환
        results = convert_index_to_text(results, END_TOKEN)

        print(f'Q: {questions[idx]}')
        print(f'A: {results}\n')
        print()

processing epoch: 1...
Epoch 1/10
126/126 [==============================] - ETA: 0s - loss: 3.0061 - acc: 0.7012
Epoch 1: loss improved from inf to 3.00615, saving model to model/seq2seq.ckpt
126/126 [==============================] - 9s 24ms/step - loss: 3.0061 - acc: 0.7012
Epoch 2/10
124/126 [============================>.] - ETA: 0s - loss: 1.7624 - acc: 0.7385
Epoch 2: loss improved from 3.00615 to 1.76297, saving model to model/seq2seq.ckpt
126/126 [==============================] - 2s 15ms/step - loss: 1.7630 - acc: 0.7384
Epoch 3/10
126/126 [==============================] - ETA: 0s - loss: 1.6354 - acc: 0.7568
Epoch 3: loss improved from 1.76297 to 1.63541, saving model to model/seq2seq.ckpt
126/126 [==============================] - 2s 16ms/step - loss: 1.6354 - acc: 0.7568
Epoch 4/10
126/126 [==============================] - ETA: 0s - loss: 1.5666 - acc: 0.7590
Epoch 4: loss improved from 1.63541 to 1.56664, saving model to model/seq2seq.ckpt
126/126 [=====================

#### **4) 챗봇 구현 및 성능 확인**
훈련된 Seq2Seq 모델을 가지고 간단한 대화를 할 수 있는 코드를 만들어보도록하겠습니다.</br>
만들어진 챗봇에 다양한 질문을 넣어보면서 모델이 어떤 문장을 생성하는지 확인해봅시다.

In [ ]:
#챗봇에서 입력한 문장을 모델에 넣을 수 있는 형태로 변환하는 함수
def question_to_input(sentence):
    sentence = clean_and_morph(sentence)
    question_sequence = tokenizer.texts_to_sequences([sentence])
    question_pad = pad_sequences(question_sequence, maxlen= max_len, truncating='post', padding='post')
    return question_pad

#간단한 챗봇 구현 -> q를 누르면 종료됩니다.
def run_chatbot(question):
    question_inputs = question_to_input(question)
    results = make_prediction(seq2seq, question_inputs)
    results = convert_index_to_text(results, END_TOKEN)
    return results

while True:
    user_input = input('대화를 입력하세요\n')
    if user_input =='q':
        break
    print('answer: {}'.format(run_chatbot(user_input)))

## **2. Attention**
- 어텐션 메커니즘은 자연어처리 분야에서 대세 모듈로 사용되고 있는 트랜스포머의 기반이 되는 메커니즘입니다.</br>
- 트랜스포머를 알아보기에 앞서, 어텐션 메커니즘의 이론에 대해 알아보고 Seq2Seq 모델에 어텐션을 적용해보도록 하겠습니다.

### **1-1. Seq2Seq의 한계점**
RNN에 기반한 Seq2Seq 모델의 인코더, 디코더 구조는 문장 생성 분야에 큰 발전을 가져다 주었지만, 다양한 문제점을 가지고 있었습니다.</br>
1. 하나의 컨텍스트 벡터에 입력 데이터의 **모든 정보를 압축**해서 디코더에 전달하므로 정보의 손실이 발생하게 됩니다.</br>
2. RNN을 기반으로하기 때문에 기울기 소실 문제가 발생합니다.</br>
3. RNN을 기반으로하기 때문에 순차적으로 데이터를 처리하므로 병렬처리가 불가능합니다.

Seq2Seq 모델이 등장했던 시기에는 하드웨어의 성능 문제 때문에 하나의 컨텍스트 벡터에 모든 정보를 압축해야했지만, 시대가 지나면서 하드웨어가 발전함에 따라 인코더의 모든 출력 정보를 하나의 컨텍스트에 담지 않아도 충분히 전달할 수 있게 되었습니다.

### **1-2. Seq2Seq with Attention**
어텐션 메커니즘을 적용한 시퀀스-투-시퀀스 모델은 디코더가 인코더의 모든 출력을 참고합니다.
<img src="https://user-images.githubusercontent.com/45377884/86040873-b942d800-ba7f-11ea-9f59-ee23923f777e.gif" alt="seq2seq_7" width="800" /></br>
그렇다면 디코더에서 각 인코더의 출력값을 어떻게 참고하게 될까요? 더 자세하게 알아보도록 하겠습니다.

### **1-3. Attention Score**
디코더에서는 지금 생성하려는 단어와 가장 연관성이 높은 인코더의 출력값을 찾아내기 위해 어텐션 스코어를 계산하게 됩니다.</br>
디코더는 어텐션 스코어가 가장 높은 인코더의 출력값을 가장 많이 반영하여 새로운 단어를 생성합니다.</br>

- **쿼리(Query), 키(Key), 벨류(Value)**</br>
어텐션 스코어를 계산할 때 가장 중요한 개념은 쿼리, 키, 벨류 벡터가 무엇인지 이해하는 것입니다.</br>
디코더는 **자신이 만들려는 단어(쿼리 벡터)**가 **기존의 단어들 중에서 어떤 것과 가장 비슷한지(키 벡터)** 확인하고 **그 단어의 값(벨류 벡터)**을 참고하여 새로운 단어를 생성합니다.</br>

어텐션을 사용하는 Seq2Seq 모델의 쿼리, 키, 벨류 벡터는 다음과 같습니다.
- 쿼리 : 디코더의 hidden state 벡터
- 키 : 인코더의 hidden state 백터
- 벨류 : 인코더의 hidden state 벡터

<img src="https://i.imgur.com/gNcbamV.png" title="source: imgur.com" width="800" /></a>

### **1-4. 모델 구현**
기존의 Seq2Seq 모델에 어텐션 메커니즘을 추가해서 구현해보도록 하겠습니다.

#### **1) 인코더(Encoder)**
하나의 컨텍스트 벡터를 생성했던 Seq2Seq 인코더와 다르게, 어텐션 인코더는 모든 인코더의 출력을 그대로 출력하도록 구현합니다.

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        '''
        Seq2Seq의 인코더입니다.

        Args:
            units (int) : 인코더 내부 lstm의 노드 수.
            vocab_size (int) : 임베딩 행렬의 단어 수. 없는 단어가 있으면 oov가 발생할 수 있습니다.
                -> 훈련하려는 문장의 단어는 모두 포함하고 있는 것이 좋다!
            embedding_dim (int) : 임베딩 차원 수. 복잡할수록 좋을 수도 있고, 아닐 수도 있습니다. 차원이 크면 보통 표현력이 좋다. but 용량이 커져서 안좋을 수 있다.
            time_steps (int) : 문장 토큰의 수. ex) 안녕하세요 조윤행입니다. -> 안녕하세요/ 조윤행/ 입니다/ -> 토큰 수 : 3개
        '''
        super().__init__()
        self.embedding = Embedding(vocab_size,
                                   embedding_dim,
                                   input_length=time_steps)
        self.dropout = Dropout(0.2)
        # (attention) return_sequences=True 추가
        self.lstm = LSTM(units,
                         return_state=True,
                         return_sequences=True
                         )

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x)
        # (attention) x return 추가
        return x, [hidden_state, cell_state]

#### **2) 디코더(Decoder)**
 디코더에서 모든 인코더 출력값에 대해 어텐션 스코어를 계산하여 반영하여 결과를 출력합니다.

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        '''
        Seq2Seq의 디코더입니다.

        Args:
            units (int) : 디코더 내부 lstm의 노드 수.
            vocab_size (int) : 임베딩 행렬의 단어 수. 없는 단어가 있으면 oov가 발생할 수 있습니다.
                -> 훈련하려는 문장의 단어는 모두 포함하고 있는 것이 좋다!
            embedding_dim (int) : 임베딩 차원 수. 복잡할수록 좋을 수도 있고, 아닐 수도 있습니다. 차원이 크면 보통 표현력이 좋다. but 용량이 커져서 안좋을 수 있다.
            time_steps (int) : 문장 토큰의 수. 디코더에서는 최대로 생성할 수 있는 문장의 길이가 됩니다.
        '''
        super().__init__()
        self.embedding = Embedding(vocab_size,
                                   embedding_dim,
                                   input_length=time_steps)
        self.dropout = Dropout(0.2)
        self.lstm = LSTM(units,
                         return_state=True,
                         return_sequences=True)
        #(attention) 어텐션 추가
        self.attention = Attention()
        self.dense = Dense(vocab_size, activation='softmax')

    def call(self, inputs, initial_state):
        # (attention) encoder_inputs 추가
        encoder_inputs, decoder_inputs = inputs
        x = self.embedding(decoder_inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)

        # (attention) query_vector, attention_matrix 추가
        # 이전 hidden_state의 값을 concat으로 만들어 query_vector를 생성합니다.
        query_vector = tf.concat([initial_state[0][:, tf.newaxis, :],
                               x[:, :-1, :]], axis=1)
        # query_vector와 인코더에서 나온 출력 값들로 attention을 구합니다.
        attention_matrix = self.attention([query_vector, encoder_inputs])
        # 위에서 구한 attention_matrix와 decoder의 출력 값을 concat 합니다.
        x = tf.concat([x, attention_matrix], axis=-1)

        x = self.dense(x)
        return x, hidden_state, cell_state

#### **3) Seq2Seq with Attention**
어텐션을 적용한 인코더와 디코더를 사용하여 seq2seq 모델을 구현합니다.

In [ ]:
class Seq2seq_with_Attention(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
        """
        어텐션이 적용된 Seq2Seq 모델입니다. 인코더와 디코더를 선언합니다.
        """
        super().__init__()
        self.start_token = start_token
        self.end_token = end_token
        self.time_steps = time_steps

        self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
        self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)

    def call(self, inputs, training=True):
        """
        선언한 인코더와 디코더를 하나로 연결하여 Seq2Sqe with Attention 파이프라인을 구현합니다.

        Args:
            inputs : 문장의 단어 인덱스로 이루어진 데이터.
            training : True인 경우 교사강요를 사용하여 디코더의 입력값에 정답을 넣어주며, False인 경우 디코더의 출력 데이터를 입력으로 넣어주게 됩니다.
        """
        if training:
            encoder_inputs, decoder_inputs = inputs
            #(attention) 인코더가 context vector뿐만 아니라 모든 출력값을 만들도록 수정
            encoder_outputs, context_vector = self.encoder(encoder_inputs)
            #(attention) 디코더가 인코더의 모든 출력값을 받도록 수정
            decoder_outputs, _, _ = self.decoder((encoder_outputs, decoder_inputs), context_vector)
            return decoder_outputs
        else:
            #(attention) 인코더가 context vector뿐만 아니라 모든 출력값을 만들도록 수정
            encoder_outputs, context_vector = self.encoder(inputs)
            target_seq = tf.constant([[self.start_token]], dtype=tf.float32)
            results = tf.TensorArray(tf.int32, self.time_steps)

            for i in tf.range(self.time_steps):
                #디코더가 인코더의 모든 출력값을 받도록 수정
                decoder_output, decoder_hidden, decoder_cell = self.decoder((encoder_outputs, target_seq),
                                                                            context_vector)
                decoder_output = tf.cast(tf.argmax(decoder_output, axis= -1),
                                         dtype=tf.int32)
                decoder_output = tf.reshape(decoder_output, shape=(1, 1))
                results = results.write(i, decoder_output)

                if decoder_output == self.end_token:
                    break

                target_seq = decoder_output
                context_vector = [decoder_hidden, decoder_cell]

            return tf.reshape(results.stack(), shape=(1, self.time_steps))

### **1-5. 챗봇 만들기**
어텐션 메커니즘이 적용된 Seq2Seq 모델을 사용하여 챗봇을 구현해보도록 하겠습니다.</br>
(과정은 기존 Seq2Seq와 동일하므로 데이터로드 및 전처리, 토큰화 과정은 생략하겠습니다.)

#### **1) 모델 훈련**

In [ ]:
#모델 훈련을 위해 모델을 선언합니다.
seq2seq_a = Seq2seq_with_Attention(UNITS,
                  VOCAB_SIZE,
                  EMBEDDING_DIM,
                  TIME_STEPS,
                  START_TOKEN,
                  END_TOKEN)

seq2seq_a.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['acc'])

In [ ]:
#학습된 모델을 저장하기위해 체크포인트를 지정합니다.
checkpoint_path = 'model/seq2seq_with_Attention.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only= True,
                             save_best_only=True,
                             monitor='loss',
                             verbose=1
                             )

In [ ]:
#모델 훈련
for epoch in range(NUM_EPOCHS):
    print(f'processing epoch: {epoch * 10 + 1}...')
    seq2seq_a.fit([question_pad, answer_in_pad],
                answer_out_pad,
                epochs=10,
                batch_size=BATCH_SIZE,
                callbacks=[checkpoint]
               )
    # 랜덤한 샘플 번호 추출
    samples = np.random.randint(DATA_LENGTH, size=SAMPLE_SIZE)

    # 예측 성능 테스트
    for idx in samples:
        question_inputs = question_pad[idx]
        # 문장 예측
        results = make_prediction(seq2seq_a, np.expand_dims(question_inputs, 0))

        # 변환된 인덱스를 문장으로 변환
        results = convert_index_to_text(results, END_TOKEN)

        print(f'Q: {questions[idx]}')
        print(f'A: {results}\n')
        print()

processing epoch: 1...
Epoch 1/10
122/126 [============================>.] - ETA: 0s - loss: 2.7165 - acc: 0.7043
Epoch 1: loss improved from inf to 2.70026, saving model to model/seq2seq_with_Attention.ckpt
126/126 [==============================] - 7s 13ms/step - loss: 2.7003 - acc: 0.7039
Epoch 2/10
126/126 [==============================] - ETA: 0s - loss: 1.7135 - acc: 0.7488
Epoch 2: loss improved from 2.70026 to 1.71346, saving model to model/seq2seq_with_Attention.ckpt
126/126 [==============================] - 1s 10ms/step - loss: 1.7135 - acc: 0.7488
Epoch 3/10
125/126 [============================>.] - ETA: 0s - loss: 1.6044 - acc: 0.7597
Epoch 3: loss improved from 1.71346 to 1.60460, saving model to model/seq2seq_with_Attention.ckpt
126/126 [==============================] - 1s 10ms/step - loss: 1.6046 - acc: 0.7597
Epoch 4/10
121/126 [===========================>..] - ETA: 0s - loss: 1.5394 - acc: 0.7611
Epoch 4: loss improved from 1.60460 to 1.53602, saving model to mode

#### **2) 챗봇 구현 및 성능 확인**
어텐션을 적용한 모델로 챗봇을 만들어 확인해보도록 하겠습니다.

In [ ]:
#sec2sec에서 만들어놓은 챗봇 실행 함수 run_chatbot를 사용합니다.
while True:
    user_input = input('대화를 입력하세요\n')
    if user_input =='q':
        break
    print('answer: {}'.format(run_chatbot(user_input)))

대화를 입력하세요
배고파
answer: 얼른 맛 난 음식 드세요 
대화를 입력하세요
추워
answer: 찬물 샤워 를 해보세요 
대화를 입력하세요
q


## **3. Transformer**
트랜스포머 모델은 자연어 처리에서 가장 많이 사용되고 있는 핵심 모델입니다.</br>
2017년에 등장한 **'Attention is All You Need"** 논문을 통해 처음 등장한 트랜스포머 모델은</br> Seq2Seq처럼 인코더, 디코더 구조는 그대로 가져가면서도</br> RNN과 같은 순환 신경망을 사용하지않고 어텐션만 사용하여 병렬 처리가 가능하게 되었습니다.

### **1-1. 모델 구조**
트랜스포머 모델의 전체적인 구조는 다음과 같습니다.</br>
트랜스포머는 6개의 인코더와 6개의 디코더를 사용합니다.</br>
<img src="http://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png" alt="positional_encoding" width="700" /></br>
인코더와 디코더의 구조를 더 자세하게 나타내면 아래와 같습니다.</br>
인코더와 디코더는 다양한 형태의 Attention과 Feed Forward 및 Add & Norm 층으로 구성된 것을 확인할 수 있습니다.</br>
<img src="https://miro.medium.com/max/1400/1*BHzGVskWGS_3jEcYYi6miQ.png" alt="positional_encoding" width="550" /></br>


#### **1) Positional Encoding**
트랜스포머는 기존의 Seq2Seq와 다르게 RNN과 같은 순환신경망을 사용하지 않기 때문에 **모든 단어의 정보를 한번에 받습니다.**</br>
<img src="https://i.imgur.com/TN5xdwH.png" alt="positional_encoding" width="550" /></br>
따라서 단어의 위치를 확인할 수 없기 때문에 위치 인코딩 과정을 거쳐 단어의 위치를 표시해줍니다.

다양한 위치 인코딩 방법이 있지만, 트랜스포머에서는 **sin**과 **cosin**함수를 사용하여 각 단어의 위치를 표시합니다.</br>
$$
\begin{aligned}
\text{PE}_{\text{pos},2i} &= \sin \bigg(\frac{\text{pos}}{10000^{2i/d_{\text{model}}}}\bigg) \\
\text{PE}_{\text{pos},2i+1} &= \cos \bigg(\frac{\text{pos}}{10000^{2i/d_{\text{model}}}}\bigg)
\end{aligned}
$$</br>
<img src="https://i.imgur.com/PpeaXWC.png" alt="positional_encoding" width="500" />

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super().__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스(2i)에는 사인 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])

    # 배열의 홀수 인덱스(2i+1)에는 코사인 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:, 0::2] = sines
    angle_rads[:, 1::2] = cosines
    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    print(pos_encoding.shape)
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

#### **2) 인코더(Encoder)**
인코더는 크게 **멀티헤드 어텐션(Multi-Head Attention)**과 **Feed Forward** 층으로 구성되어 있습니다.</br>
<img src="https://i.imgur.com/0LjTimw.png" alt="positional_encoding" width="550" /></br>

**1. Self Attention & Multi-Head Attention**</br>
트랜스포머에서 사용하는 Attention의 구조는 다음과 같습니다.</br>
트랜스포머의 인코더와 디코더는 **여러개의 어텐션을 계산한 다음에 하나로 합하여 사용**하는 멀티 헤드 어텐션 방식을 사용합니다.
<img src="https://i.imgur.com/n30tcnt.png" alt="positional_encoding" width="700" /></br>
셀프 어텐션의 계산 과정은 다음과 같습니다.</br>

**1. 쿼리, 키, 벨류 벡터를 계산합니다**</br>
퀴리,키, 벨류 벡터는 입력데이터에 각각의 가중치가 곱해져 만들어집니다.</br>
<img src="http://jalammar.github.io/images/t/self-attention-matrix-calculation.png" alt="transformer_12" width="400" /></br>
<img src="http://jalammar.github.io/images/xlnet/self-attention-1.png" alt="transformer_15" width="600" /></br>
**2. 쿼리와 키 벡터를 곱하여 특정 단어가 다른 단어들에 대해 어느정도의 연관성을 가지고 있는지 나타내는 어텐션 스코어를 계산합니다.**</br>
<img src="https://i.imgur.com/9gdExlq.png" alt="positional_encoding" width="400" />
<img src="http://jalammar.github.io/images/t/transformer_self-attention_visualization.png" alt="self_attention_visualization" width="350" /></br>
<img src="http://jalammar.github.io/images/xlnet/self-attention-2.png" alt="transformer_15" width="600" /></br>
**3. 계산된 Attention Score와 벨류 벡터를 곱하여 컨텍스트 벡터를 계산합니다.**</br>
<img src="http://jalammar.github.io/images/xlnet/self-attention-3.png" alt="transformer_15" width="600" />

셀프 어텐션의 계산 과정을 식으로 표현하면 다음과 같습니다.</br>
<img src="http://jalammar.github.io/images/t/self-attention-matrix-calculation-2.png" alt="transformer_13" width="700" />

In [ ]:
def scaled_dot_product_attention(query, key, value, mask):
  # query 크기 : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  # key 크기 : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
  # value 크기 : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
  # padding_mask : (batch_size, 1, 1, key의 문장 길이)

  # Q와 K의 곱. 어텐션 스코어 행렬.
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 스케일링
  # dk의 루트값으로 나눠준다.
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 마스킹. 어텐션 스코어 행렬의 마스킹 할 위치에 매우 작은 음수값을 넣는다.
  # 매우 작은 값이므로 소프트맥스 함수를 지나면 행렬의 해당 위치의 값은 0이 된다.
  if mask is not None:
    logits += (mask * -1e9)

  # 소프트맥스 함수는 마지막 차원인 key의 문장 길이 방향으로 수행된다.
  # attention weight : (batch_size, num_heads, query의 문장 길이, key의 문장 길이)
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # output : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  output = tf.matmul(attention_weights, value)

  return output, attention_weights

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    # d_model을 num_heads로 나눈 값.
    # 논문 기준 : 64
    self.depth = d_model // self.num_heads

    # WQ, WK, WV에 해당하는 밀집층 정의
    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    # WO에 해당하는 밀집층 정의
    self.dense = tf.keras.layers.Dense(units=d_model)

  # num_heads 개수만큼 q, k, v를 split하는 함수
  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # 1. WQ, WK, WV에 해당하는 밀집층 지나기
    # q : (batch_size, query의 문장 길이, d_model)
    # k : (batch_size, key의 문장 길이, d_model)
    # v : (batch_size, value의 문장 길이, d_model)
    # 참고) 인코더(k, v)-디코더(q) 어텐션에서는 query 길이와 key, value의 길이는 다를 수 있다.
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 2. 헤드 나누기
    # q : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    # k : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
    # v : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
    # (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
    # (batch_size, query의 문장 길이, num_heads, d_model/num_heads)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 4. 헤드 연결(concatenate)하기
    # (batch_size, query의 문장 길이, d_model)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 5. WO에 해당하는 밀집층 지나기
    # (batch_size, query의 문장 길이, d_model)
    outputs = self.dense(concat_attention)

    return outputs

**2. Layer Normalization & Skip Connection**</br>
인코더와 디코더에는 모든 출력에 대해 **Layer Normalization과 Skip connection**이 적용되어 있습니다.(Add & Norm)</br>
- **Layer Normalization**</br>
배치 사이즈 단위로 정규화를 진행했던 배치 정규화와는 다르게, 층 정규화는 벡터 크기 단위로 정규화를 진행합니다. 배치 정규화와 방식은 다르지만 학습이 더 잘 되게 만들어준다는 공통점을 가지고 있습니다.
- **Skip Connection**</br>
특정 층을 거칠 때, 해당 층을 지나지 않은 상태의 데이터와 다시 합쳐줌으로써 역전파 과정에서 정보가 손실되는 것을 막아주는 역할을 합니다.</br>
<img src="https://miro.medium.com/max/1400/1*BHzGVskWGS_3jEcYYi6miQ.png" alt="positional_encoding" width="550" /></br>
<img src="https://i.imgur.com/LJ8QDgY.png" alt="transformer_13" width="700" />


**3. Feed Forward**</br>
Feed forward 층은 활성화 함수로 Relu를 사용하는 2층 fully connected 신경망입니다.</br>


$$
 \text{FFNN}(x) = \max(0, W_1x + b_1) W_2 +b_2
$$

In [ ]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, key의 문장 길이)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [ ]:
def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 멀티-헤드 어텐션 (첫번째 서브층 / 셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': padding_mask # 패딩 마스크 사용
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 포지션 와이즈 피드 포워드 신경망 (두번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
def encoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 인코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

#### **3) 디코더(Decoder)**
디코더는 크게 **마스크 멀티헤드 어텐션(Masked Multi-Head Attention)**과 **멀티헤드 어텐션(Multi-Head Attention)**, **Feed Forward** 층으로 구성되어 있습니다.</br>
<img src="https://i.imgur.com/DA1FTkw.png" alt="positional_encoding" width="550" /></br>

**1. Masked Multi-Head Attention**</br>
트랜스포머는 데이터를 한번에 받는다고 이야기했었습니다.</br>

**디코더의 입력 데이터가 그대로 한번에 들어오게 된다면 어떤 문제가 발생할 수 있을까요?**</br>

입력 데이터를 디코더에 그대로 전달한다면, 다음에 올 단어에 대한 정보를 입력 데이터를 통해 얻을 수 있게 되어 일종의 데이터 누수와 같은 문제가 발생할 수 있습니다!</br>

이를 막기 위해 디코더에서는 현재 생성하려는 단어 위치 이후의 정보는 확인할 수 없는 데이터로 바꿔주는 마스킹 단계를 거치게 됩니다.</br>
<img width="500" alt="Masked_Self-Attention_ex" src="http://jalammar.github.io/images/xlnet/transformer-decoder-block-self-attention-2.png">

마스킹은 어텐션 단계에서 소프트맥스 함수를 사용한다는 것을 이용해서 - 무한대 값을 곱해주는 방법을 사용합니다.</br>
<img width="600" alt="masked_1" src="http://jalammar.github.io/images/gpt2/transformer-attention-mask.png">

<img width="600" alt="masked_2" src="http://jalammar.github.io/images/gpt2/transformer-attention-masked-scores-softmax.png">

In [ ]:
# 디코더의 첫번째 서브층(sublayer)에서 미래 토큰을 Mask하는 함수
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x) # 패딩 마스크도 포함
  return tf.maximum(look_ahead_mask, padding_mask)

**2. Multi-Head Attention(디코더)**</br>
<img src="https://i.imgur.com/qMfpeXB.png" alt="positional_encoding" width="550" /></br>
디코더의 멀티 헤드 어텐션은 인코더의 셀프 어텐션과 조금의 차이점을 가지고 있습니다.</br>

인코더의 셀프 어텐션은 쿼리, 키, 벨류 벡터를 모두 직접 생성했지만,</br>
**디코더 어텐션에서는 키,벨류 벡터로 인코더의 키,벨류 벡터를 가져와 사용합니다.**</br>(seq2seq에서 사용된 어텐션 원리와 동일)</br>
<img width="700" alt="Encoder-Decoder_Attention_gif" src="http://jalammar.github.io/images/t/transformer_decoding_1.gif">

In [ ]:
def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

  # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 멀티-헤드 어텐션 (첫번째 서브층 / 마스크드 셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': look_ahead_mask # 룩어헤드 마스크
      })

  # 잔차 연결과 층 정규화
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 멀티-헤드 어텐션 (두번째 서브층 / 디코더-인코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1, 'key': enc_outputs, 'value': enc_outputs, # Q != K = V
          'mask': padding_mask # 패딩 마스크
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 포지션 와이즈 피드 포워드 신경망 (세번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
def decoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')

  # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 디코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

### **1-2. 모델 구현**

In [ ]:
def transformer(vocab_size, num_layers, dff,
                d_model, num_heads, dropout,
                name="transformer"):

  # 인코더의 입력
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 디코더의 입력
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더의 패딩 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더의 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 디코더의 패딩 마스크(두번째 서브층)
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더의 출력은 enc_outputs. 디코더로 전달된다.
  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[inputs, enc_padding_mask]) # 인코더의 입력은 입력 문장과 패딩 마스크

  # 디코더의 출력은 dec_outputs. 출력층으로 전달된다.
  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 다음 단어 예측을 위한 출력층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

### **1-3. 챗봇 만들기**
트랜스포머 모델을 사용하여 챗봇을 구현해보도록 하겠습니다.</br>

#### **1) 데이터 전처리**

In [ ]:
questions = []
for sentence in corpus['Q']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

answers = []
for sentence in corpus['A']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

#### **2) 토큰화**

In [ ]:
# 서브워드텍스트인코더를 사용하여 질문과 답변을 모두 포함한 단어 집합(Vocabulary) 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)

# 시작 토큰과 종료 토큰에 대한 정수 부여.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2

In [ ]:
# 최대 길이를 정의
MAX_LENGTH = 30

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2)

  # 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs

questions, answers = tokenize_and_filter(questions, answers)

In [ ]:
# 텐서플로우 dataset을 이용하여 셔플(shuffle)을 수행하되, 배치 크기로 데이터를 묶는다.
# 또한 이 과정에서 교사 강요(teacher forcing)을 사용하기 위해서 디코더의 입력과 실제값 시퀀스를 구성한다.
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {
        'outputs': answers[:, 1:]  # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다.
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

#### **3) 모델 훈련**

In [ ]:
# Hyper-parameters
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

(1, 8180, 256)
(1, 8180, 256)


In [ ]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
MAX_LENGTH = 30

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [ ]:
EPOCHS = 50

model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
185/185 [==============================] - 15s 48ms/step - loss: 1.9425 - accuracy: 0.0395
Epoch 2/50
185/185 [==============================] - 10s 56ms/step - loss: 1.5768 - accuracy: 0.0665
Epoch 3/50
185/185 [==============================] - 13s 68ms/step - loss: 1.3470 - accuracy: 0.0682
Epoch 4/50
185/185 [==============================] - 9s 48ms/step - loss: 1.2480 - accuracy: 0.0732
Epoch 5/50
185/185 [==============================] - 9s 48ms/step - loss: 1.1701 - accuracy: 0.0775
Epoch 6/50
185/185 [==============================] - 10s 52ms/step - loss: 1.0894 - accuracy: 0.0834
Epoch 7/50
185/185 [==============================] - 9s 47ms/step - loss: 1.0014 - accuracy: 0.0913
Epoch 8/50
185/185 [==============================] - 9s 48ms/step - loss: 0.9021 - accuracy: 0.1015
Epoch 9/50
185/185 [==============================] - 9s 48ms/step - loss: 0.7961 - accuracy: 0.1132
Epoch 10/50
185/185 [==============================] - 11s 60ms/step - loss: 0.6860 - a

#### **4) 챗봇 구현 및 성능 확인**
트랜스포머 모델로 챗봇을 만들어 확인해보도록 하겠습니다.

In [ ]:
def preprocess_sentence(sentence):
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

In [ ]:
def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # 현재(마지막) 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 마지막 시점의 예측 단어를 출력에 연결한다.
    # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)


def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  return predicted_sentence

In [ ]:
#간단한 챗봇 구현 -> q를 누르면 종료됩니다.
while True:
    user_input = input('대화를 입력하세요\n')
    if user_input =='q':
        break
    print('answer: {}'.format(predict(user_input)))

대화를 입력하세요
안녕하세요
answer: 안녕하세요 .
대화를 입력하세요
배고파
answer: 뭐 좀 챙겨드세요 .
대화를 입력하세요
나 오늘 아파
answer: 맘 고생 많았어요 .
대화를 입력하세요
q
